### Load and Investigate Data

In [1]:
import pandas as pd

In [ ]:
pl_matches = pd.read_csv("pl_matches.csv", index_col = 0) # preserve index structure
pl_matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2,0,Ipswich Town,...,Match Report,NaN,18.0,5.0,14.8,0.0,0,0,2025,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,...,Match Report,NaN,19.0,8.0,13.6,1.0,0,0,2025,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3,0,Manchester Utd,...,Match Report,NaN,11.0,3.0,13.4,0.0,0,0,2025,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,...,Match Report,NaN,14.0,5.0,14.9,0.0,0,0,2025,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,Bournemouth,...,Match Report,NaN,19.0,12.0,16.6,0.0,0,0,2025,Liverpool


In [3]:
pl_matches.shape

(2280, 28)

### Investigate Potential Missing Data

In [6]:
print(f"Total matches over three seasons should be: {20 * 38 * 3}") # 20 teams, 38 matches/team, 3 seasons

# See how many matches each team has played
pl_matches["team"].value_counts()

Total matches over three seasons should be: 2280


team
Liverpool                   114
Brentford                   114
Tottenham Hotspur           114
Wolverhampton Wanderers     114
Manchester United           114
West Ham United             114
Arsenal                     114
Crystal Palace              114
Fulham                      114
Everton                     114
Bournemouth                 114
Brighton and Hove Albion    114
Nottingham Forest           114
Aston Villa                 114
Newcastle United            114
Chelsea                     114
Manchester City             114
Leicester City               76
Southampton                  76
Ipswich Town                 38
Luton Town                   38
Burnley                      38
Sheffield United             38
Leeds United                 38
Name: count, dtype: int64

Data looks good:
- 3 teams get pulled up (38 matches)
- 2 teams played in the PL 2/3 past seasons (76 matches)
- The rest played in the PL for all three seasons 

### Clean Data for ML

In [7]:
# ML algos only work with numeric data
pl_matches.dtypes

date              object
time              object
comp              object
round             object
day               object
venue             object
result            object
gf                 int64
ga                 int64
opponent          object
xg               float64
xga              float64
poss             float64
attendance       float64
captain           object
formation         object
opp formation     object
referee           object
match report      object
notes            float64
sh               float64
sot              float64
dist             float64
fk               float64
pk                 int64
pkatt              int64
season             int64
team              object
dtype: object